In [76]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [77]:
r = requests.get(constants.DSHS_XHR_URL, verify=False)
r.raise_for_status()

/Users/dharbin/work/texascovid19/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services5.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [78]:
county_data = r.json()['features']

In [79]:
norm_county_data = []
for county in county_data:
        norm_county_data.append({
            'County of Residence': county['attributes']['County'],
            'Deaths': county['attributes']['Fatalities'],
            'Number of Cases': county['attributes']['Positive'],
            'Recoveries': county['attributes']['Recoveries'],
            'Active': county['attributes']['Active']
        })

In [80]:
df = pd.DataFrame(data=norm_county_data)

In [81]:
df

,County of Residence,Deaths,Number of Cases,Recoveries,Active
0,Harris,188,8621,3315,4918
1,Dallas,153,6602,2819,3392
2,Tarrant,114,4076,821,2880
3,Travis,72,2288,782,1382
4,Bexar,58,1976,1050,835
...,...,...,...,...,...
214,Knox,0,1,1,0
215,Mitchell,0,1,1,0
216,Motley,0,1,1,0
217,Reeves,0,1,0,1


In [82]:
df.set_index("County of Residence", inplace=True)

In [83]:
today = date.today().isoformat()

In [84]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [85]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [86]:
df_copy = df.copy()

In [87]:
df_copy.rename(columns={"Number of Cases": today}, inplace=True)

In [88]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [89]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [90]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [91]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [92]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)

Also for deaths

In [93]:
df_timeseries = pd.read_csv(f"{constants.DEATHS_TIMESERIES_CSV}").set_index("County of Residence")

In [94]:
df_copy = df.copy()

In [95]:
df_copy.rename(columns={"Deaths": today}, inplace=True)

In [96]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [97]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [98]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [99]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [100]:
df_timeseries.to_csv(constants.DEATHS_TIMESERIES_CSV, header=True, index=True)